In [3]:
import pandas as pd
from geojson import Feature, Point, MultiPoint, Polygon, FeatureCollection
import numpy as np
from scipy.spatial import ConvexHull
from scipy import spatial
import geopandas as gpd
from optimization2 import getServiceArea

In [178]:
buildings_all = gpd.read_file('all_buildings_SF_footprint_elev_2.shp')
business = gpd.read_file('business_SF_WGS.shp')
residential_buildings = gpd.read_file('residential_buildings_footprint_2.shp')

# Create Tree

In [179]:
buildings_all['OBJECTID'] = buildings_all.index
X_lat_lon = list(zip(buildings_all['y_lat'],buildings_all['x_lon']))
tree = spatial.KDTree(X_lat_lon)
X_lat_lon_emp = list(zip(business['lat'],business['lng']))

# Find closest building

In [180]:
location = 0
for item in X_lat_lon_emp:
    dist, index_select = tree.query(item, k=1)
    close_to = int(buildings_all.iloc[int(index_select)]['OBJECTID_1'])
    business.loc[int(location),'close_id'] = close_to
    location+=1
    

# Dissolve by closest building and add employee count

In [181]:
new_business = business[['geometry','close_id', 'EMPNUM']]
employee_count = new_business.dissolve(by='close_id', aggfunc='sum')

In [182]:
employee_buildings = buildings_all.join(employee_count, on='OBJECTID_1', how = 'inner', rsuffix='_business')

In [183]:
employee_buildings = employee_buildings.drop('geometry_business',1)

In [184]:
employee_buildings['index_vals'] = employee_buildings['OBJECTID']
employee_buildings['OBJECTID'] = employee_buildings['OBJECTID_1']

# Merge with residential buildings

In [185]:
frames = [residential_buildings, employee_buildings]
result = pd.concat(frames)

In [186]:
subset_result = result[['OBJECTID', 'geometry' ,'SUM_pop', 'EMPNUM']]

# Dissolve based on OBJECTID and sum population and employee count

In [187]:
combined_buildings_sum = subset_result.dissolve(by='OBJECTID', aggfunc='sum')

In [188]:
combined_buildings = buildings_all.join(combined_buildings_sum, on='OBJECTID_1', how = 'inner', rsuffix='_combined')

In [189]:
combined_buildings['residential_pop'] = combined_buildings['SUM_pop']

In [190]:
combined_buildings['SUM_pop'] = combined_buildings.residential_pop.fillna(0) + combined_buildings.EMPNUM.fillna(0)

In [191]:
new_combined_buildings = combined_buildings[['num_floor', 'Area_m2', 'x_lon', 'y_lat', 'SUM_pop', 'residential_pop', 'EMPNUM', 'ELEV_treat']]

In [192]:
new_combined_buildings['lat_lon'] = new_combined_buildings.apply(lambda row: (row['y_lat'], row['x_lon']), axis=1)

C:\Users\Olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [193]:
new_combined_buildings.to_csv('combined_buildings_2.csv')

# Run for grid locations

In [14]:
def make_grid(bounding_box, xcell, ycell):
    xmax, xmin, ymax, ymin = bounding_box
    xgrid = np.linspace(xmin, xmax, xcell)
    ygrid = np.linspace(ymin, ymax, ycell)
    mX, mY = np.meshgrid(xgrid, ygrid)
    ngridX = mX.reshape(xcell*ycell, 1);
    ngridY = mY.reshape(xcell*ycell, 1);
    return np.concatenate((ngridX, ngridY), axis=1)

def make_grid_points(n_regen, nx, ny):
    bounding_box = [37.714, 37.803, -122.373, -122.508]
    Xgrid = make_grid(bounding_box, nx, ny)
    grid_coord=[]
    count = 0
    for row in Xgrid:
        if count<n_regen:
            xgrid=row[0]
            ygrid=row[1]
            count = count + 1
            grid_coords = (xgrid, ygrid)
            grid_coord.append(grid_coords)
    return grid_coord

In [15]:
grid_points = make_grid_points(100, 10, 10)
path = 'document.csv'

In [47]:
for item in grid_points:
    polygon, points = getServiceArea(item, path, 9.5, -0.3, 0, 0, 8, -0.1, 0, 0)

begin time 0.0 
tree time 0.03302288055419922 
init query 0.009006500244140625 
init dataf 0.015511512756347656 
init time 0.0360255241394043 
Grpah time 1.1759212017059326 
MST time 0.0045032501220703125 
calc time 7.495804786682129 
out time 0.924752950668335 
loop time 9.818294525146484 
begin time 0.0 
tree time 0.04052925109863281 
init query 0.004002809524536133 
init dataf 0.009006261825561523 
init time 0.027019262313842773 
Grpah time 1.1681504249572754 
MST time 0.005001068115234375 
calc time 7.5337560176849365 
out time 0.6559689044952393 
loop time 9.506982326507568 
begin time 0.0 
tree time 0.014510393142700195 
init query 0.0030028820037841797 
init dataf 0.0060045719146728516 
init time 0.011011600494384766 
Grpah time 2.3943963050842285 
MST time 0.031136035919189453 
calc time 6.928793430328369 
out time 0.3715052604675293 
loop time 9.829406261444092 
begin time 0.0 
tree time 0.00850534439086914 
init query 0.0020017623901367188 
init dataf 0.004002809524536133 
in

In [48]:
data = pd.read_csv('document.csv')

In [49]:
data[['lat', 'lon']] = data['latlon'].str[1:-1].str.split(',', expand=True).astype(float)

In [50]:
data.to_csv('results.csv')

# Run for treatment techs

In [24]:
import json
import glob
import folium
from IPython.display import HTML
from random import randint
from matplotlib.colors import rgb2hex

In [58]:
def revert_coords(array):
    reverted_array = []
    for item in reversed(array):
        reverted_array.append(item)
    return reverted_array


def get_colored(item):
    color = '#6f786a'
    if item['properties']['accept'] == 'yes':
        color = '#f00'
    elif item['properties']['accept'] == 'no':
        color = '#000'
        
    return color

def get_radius(item):
    radius = 4
    if item['properties']['accept'] == 'yes':
        radius = 8
    elif item['properties']['accept'] == 'no':
        radius = 7
        
    return radius

In [21]:
latlon = (37.756, -122.414)
path = 'treatment.csv'

In [79]:
a, b, c, d = (9.5, -0.3, 0, 0)
e, f, g, h = (8, -0.1,  0, 0)
polygon, points = getServiceArea(latlon, path, a, b, c, d, e, f, g, h)
treat_map = folium.Map(location=[37.6236, -122.3750], tiles = 'cartodbpositron')
folium.Marker(latlon).add_to(treat_map)

for item in points['features']:
    folium.CircleMarker(revert_coords(item['geometry']['coordinates']), radius=get_radius(item), color = get_colored(item), fill_color = get_colored(item)).add_to(treat_map)  
treat_map

begin time 0.0 
tree time 0.020006895065307617 
init query 0.004002809524536133 
init dataf 0.007003307342529297 
init time 0.02101588249206543 
Grpah time 9.597101926803589 
MST time 0.0436556339263916 
calc time 10.994362592697144 
out time 0.48284006118774414 
loop time 21.328917741775513 


In [81]:
a, b, c, d = (4, 1, 0, 0)
e, f, g, h = (0, 0,  0, 0)
polygon, points = getServiceArea(latlon, path, a, b, c, d, e, f, g, h)

treat_map = folium.Map(location=[37.6236, -122.3750], tiles = 'cartodbpositron')
folium.Marker(latlon).add_to(treat_map)

for item in points['features']:
    folium.CircleMarker(revert_coords(item['geometry']['coordinates']), radius=get_radius(item), color = get_colored(item), fill_color = get_colored(item)).add_to(treat_map)  
treat_map

begin time 0.0 
tree time 0.02101445198059082 
init query 0.0050048828125 
init dataf 0.007004976272583008 
init time 0.021014928817749023 
Grpah time 0.20395326614379883 
MST time 0.0005013942718505859 
calc time 3.2642924785614014 
out time 0.7181744575500488 
loop time 4.295269250869751 


In [82]:
points

{'features': [{'geometry': {'coordinates': [-122.4139648, 37.75586979],
    'type': 'Point'},
   'properties': {'SUM_pop_commercial': 0.0,
    'SUM_pop_residential': 2.70699579,
    'accept': 'start',
    'energy': 0.0,
    'houses': 1,
    'index': 59756,
    'num_floor': 2,
    'population': 2},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-122.41387140000002, 37.75590029999999],
    'type': 'Point'},
   'properties': {'SUM_pop_commercial': 0.0,
    'SUM_pop_residential': 5.332244645,
    'accept': 'no',
    'energy': 22.021000371439627,
    'houses': 1,
    'index': 59755,
    'num_floor': 4,
    'population': 2},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-122.4141304, 37.75611743], 'type': 'Point'},
   'properties': {'SUM_pop_commercial': 0.0,
    'SUM_pop_residential': 4.100018105,
    'accept': 'no',
    'energy': 19.39979008098134,
    'houses': 1,
    'index': 59759,
    'num_floor': 3,
    'population': 2},
   'type': 'Feature'},
  {'geometry': {'coordin